In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dlp-jan-2025-nppe-3/sample_submission.csv
/kaggle/input/dlp-jan-2025-nppe-3/submission.py
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00159.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00056.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00017.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00124.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00140.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00068.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00019.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00266.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00236.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00148.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00152.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00226.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00008.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00216.png
/kaggle/input/dlp-jan-2025-n

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
import os
from PIL import Image


In [15]:
class SwinIR(nn.Module):
    def __init__(self, img_size=64, window_size=8, embed_dim=60, depths=[6, 6, 6], num_heads=[6, 6, 6], mlp_ratio=2., upscale=4):
        super(SwinIR, self).__init__()
        self.upscale = upscale
        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.num_features = int(embed_dim * 2 ** (self.num_layers - 1))

        self.conv_first = nn.Conv2d(3, embed_dim, 3, 1, 1)

        self.layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(embed_dim if i == 0 else embed_dim * 2 ** (i - 1), embed_dim * 2 ** i, 3, 1, 1),
                nn.GELU(),
                nn.Conv2d(embed_dim * 2 ** i, embed_dim * 2 ** i, 3, 1, 1)
            ) for i in range(self.num_layers)
        ])

        self.conv_after_body = nn.Conv2d(self.num_features, embed_dim, 3, 1, 1)
        self.upsample = nn.Sequential(
            nn.Conv2d(embed_dim, embed_dim * upscale ** 2, 3, 1, 1),
            nn.PixelShuffle(upscale),
            nn.Conv2d(embed_dim, 3, 3, 1, 1)  # fixed here
            )

    def forward(self, x):
        x = self.conv_first(x)
        for layer in self.layers:
            x = layer(x)
        x = self.conv_after_body(x)
        x = self.upsample(x)
        return x

model = SwinIR().cuda()


In [16]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class ImageDataset(Dataset):
    def __init__(self, lr_folder, hr_folder=None, transform=None):
        self.lr_folder = lr_folder
        self.hr_folder = hr_folder
        self.transform = transform
        self.lr_images = os.listdir(lr_folder)
        if hr_folder:
            self.hr_images = os.listdir(hr_folder)

    def __len__(self):
        return len(self.lr_images)

    def __getitem__(self, idx):
        lr_image = Image.open(os.path.join(self.lr_folder, self.lr_images[idx]))
        if self.hr_folder:
            hr_image = Image.open(os.path.join(self.hr_folder, self.hr_images[idx]))
            if self.transform:
                lr_image = self.transform(lr_image)
                hr_image = self.transform(hr_image)
            return lr_image, hr_image, self.lr_images[idx]
        else:
            if self.transform:
                lr_image = self.transform(lr_image)
            return lr_image, self.lr_images[idx]

train_lr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/train/train/'
train_hr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/train/gt/'
eval_lr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/val/val/'
eval_hr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/'
test_lr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/test/'

train_dataset = ImageDataset(train_lr_folder, train_hr_folder, transform=transform)
eval_dataset = ImageDataset(eval_lr_folder, eval_hr_folder, transform=transform)
test_dataset = ImageDataset(test_lr_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)  # Reduced batch size
eval_loader = DataLoader(eval_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [17]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Reduced learning rate


In [18]:
from torch.cuda.amp import GradScaler, autocast

scaler = GradScaler()

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for lr_images, hr_images, _ in train_loader:
        lr_images = lr_images.cuda()
        hr_images = hr_images.cuda()

        optimizer.zero_grad()

        with autocast():
            outputs = model(lr_images)
            loss = criterion(outputs, hr_images)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Validation
    model.eval()
    eval_loss = 0.0
    with torch.no_grad():
        for lr_images, hr_images, _ in eval_loader:
            lr_images = lr_images.cuda()
            hr_images = hr_images.cuda()
            outputs = model(lr_images)
            loss = criterion(outputs, hr_images)
            eval_loss += loss.item()

    print(f'Evaluation Loss: {eval_loss/len(eval_loader)}')


<ipython-input-18-e6bac7b66964>:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-18-e6bac7b66964>:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [1/50], Loss: 0.025817364052489453
Evaluation Loss: 0.027572323957374738
Epoch [2/50], Loss: 0.004563730319336541
Evaluation Loss: 0.025708498405431632
Epoch [3/50], Loss: 0.0032659983934854773
Evaluation Loss: 0.024983570341076424
Epoch [4/50], Loss: 0.0025931828401458286
Evaluation Loss: 0.024761634030893667
Epoch [5/50], Loss: 0.0021822049239623473
Evaluation Loss: 0.024226714731819594
Epoch [6/50], Loss: 0.001914028221210095
Evaluation Loss: 0.02429040971754202
Epoch [7/50], Loss: 0.0018247853542614177
Evaluation Loss: 0.024215398217315106
Epoch [8/50], Loss: 0.0016817879902588924
Evaluation Loss: 0.024278209377914223
Epoch [9/50], Loss: 0.0016793311207508163
Evaluation Loss: 0.02435416596204932
Epoch [10/50], Loss: 0.0016377112487354567
Evaluation Loss: 0.02499915453702656
Epoch [11/50], Loss: 0.0016886121538780387
Evaluation Loss: 0.024041117724976434
Epoch [12/50], Loss: 0.0016062504608547214
Evaluation Loss: 0.024443817185933022
Epoch [13/50], Loss: 0.0016457810869780016


In [19]:
model.eval()
output_folder = '/kaggle/working/test_predictions'
os.makedirs(output_folder, exist_ok=True)

with torch.no_grad():
    for lr_images, filenames in test_loader:
        lr_images = lr_images.cuda()
        outputs = model(lr_images)
        outputs = outputs.cpu()

        for i, img in enumerate(outputs):
            img = img.permute(1, 2, 0).numpy()
            img = (img * 0.5) + 0.5  # Unnormalize
            img = (img * 255).astype(np.uint8)
            Image.fromarray(img).save(os.path.join(output_folder, filenames[i]))


In [20]:
import os
import numpy as np
import pandas as pd
from PIL import Image

def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L') 
            image_array = np.array(image).flatten()[::8]
            # Replace 'test_' with 'gt_' in the ID
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

folder_path = '/kaggle/working/test_predictions'
output_csv = 'submission.csv'
images_to_csv(folder_path, output_csv)

Successfully saved to submission.csv
